<a href="https://colab.research.google.com/github/patrickbryant1/EvoBind/blob/master/EvoBind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 EvoBind
In silico directed evolution of peptide binders with [AlphaFold2](https://www.nature.com/articles/s41586-021-03819-2)
\
**EvoBind** designs peptide binders towards user-specified target residues using only sequence information. EvoBind accounts for adaptation of the receptor interface structure to the peptide design during optimization. This consideration of flexibility is crucial for binding.

<img src="https://github.com/patrickbryant1/EvoBind/blob/master/EvoBind.png" height="200" style="height:240px">

AlphaFold2 is available under the [Apache License, Version 2.0](http://www.apache.org/licenses/LICENSE-2.0) and so is EvoBind, which is a derivative thereof.  \
The AlphaFold2 parameters are made available under the terms of the [CC BY 4.0 license](https://creativecommons.org/licenses/by/4.0/legalcode) and have not been modified.
\
**You may not use these files except in compliance with the licenses.**